<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>


# Pretraining word and entity embeddings
This notebook trains word embeddings and entity embeddings for DKN initializations.

In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import time
from utils.general import *
import numpy as np
import pickle
from utils.task_helper import *

In [2]:
class MySentenceCollection:
    def __init__(self, filename):
        self.filename = filename
        self.rd = None

    def __iter__(self):
        self.rd = open(self.filename, 'r', encoding='utf-8', newline='\r\n')
        return self

    def __next__(self):
        line = self.rd.readline()
        if line:
            return list(line.strip('\r\n').split(' '))
        else:
            self.rd.close()
            raise StopIteration


In [3]:
InFile_dir = 'data_folder/my'
OutFile_dir = 'data_folder/my/pretrained-embeddings'
OutFile_dir_KG = 'data_folder/my/KG'
OutFile_dir_DKN = 'data_folder/my/DKN-training-folder'

We use word2vec algorithm implemented in Gensim (https://radimrehurek.com/gensim/models/word2vec.html) to generate word embeddings.

In [4]:
def train_word2vec(Path_sentences, OutFile_dir):     
    OutFile_word2vec = os.path.join(OutFile_dir, r'word2vec.model')
    OutFile_word2vec_txt = os.path.join(OutFile_dir, r'word2vec.txt')
    create_dir(OutFile_dir)

    print('start to train word embedding...', end=' ')
    my_sentences = MySentenceCollection(Path_sentences)
    model = Word2Vec(my_sentences, size=32, window=5, min_count=1, workers=8, iter=30)

    model.save(OutFile_word2vec)
    model.wv.save_word2vec_format(OutFile_word2vec_txt, binary=False)
    print('\tdone . ')

Path_sentences = os.path.join(InFile_dir, 'sentence.txt')

t0 = time.time()
train_word2vec(Path_sentences, OutFile_dir)
t1 = time.time()
print('time elapses: {0:.1f}s'.format(t1 - t0))

start to train word embedding... 	done . 
time elapses: 526.3s


We leverage a graph embedding model to encode entities into embedding vectors.
<img src="https://recodatasets.blob.core.windows.net/kdd2020/images%2Fkg-embedding-math.JPG" width="600">
<img src="https://recodatasets.blob.core.windows.net/kdd2020/images%2Fkg-embedding.JPG" width="600">
We use an open-source implementation of TransE (https://github.com/thunlp/Fast-TransX) for generating knowledge graph embeddings:

In [5]:
!bash ./run_transE.sh

/data/home/jialia/jialia/kdd2020tutorial/formal_02/recommenders/scenarios/KDD2020-tutorial
fatal: destination path 'Fast-TransX' already exists and is not an empty directory.
epoch 0 464878.218750
epoch 1 392123.312500
epoch 2 361906.625000
epoch 3 315392.156250
epoch 4 310050.875000
epoch 5 281908.250000
epoch 6 271810.968750
epoch 7 240873.968750
epoch 8 237960.375000
epoch 9 221742.484375


DKN take considerations of both the entity embeddings and its context embeddings.
<img src="https://recodatasets.blob.core.windows.net/kdd2020/images/context-embedding.JPG" width="600">

In [6]:
##### build context embedding
EMBEDDING_LENGTH = 32
entity_file = os.path.join(OutFile_dir_KG, 'entity2vec.vec') 
context_file = os.path.join(OutFile_dir_KG, 'context2vec.vec')   
kg_file = os.path.join(OutFile_dir_KG, 'train2id.txt')   
gen_context_embedding(entity_file, context_file, kg_file)

NameError: name 'EMBEDDING_LENGTH' is not defined

In [ ]:
load_np_from_txt(
        os.path.join(OutFile_dir_KG, 'entity2vec.vec'),
        os.path.join(OutFile_dir_DKN, 'entity_embedding.npy'),
    )
load_np_from_txt(
        os.path.join(OutFile_dir_KG, 'context2vec.vec'),
        os.path.join(OutFile_dir_DKN, 'context_embedding.npy'),
    )
format_word_embeddings(
    os.path.join(OutFile_dir, 'word2vec.txt'),
    os.path.join(InFile_dir, 'word2idx.pkl'),
    os.path.join(OutFile_dir_DKN, 'word_embedding.npy')
)
